In [488]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [489]:
dossier="/Users/gilles/Github/eCloze/"
enonce="MCLangueEcrite1ePart2"
theme="MCLangueEcrite1ePart2"

In [490]:
from ToolsMoodleXML2 import *
import ToolsMoodleXML
import yaml

In [491]:
yamlFichier=dossier+enonce+".yaml"
csvFichier=dossier+theme+".txt"
boolEX=False
titreNoEX=False

In [492]:
with codecs.open(yamlFichier, 'r',encoding="utf8") as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r',encoding="utf8") as file:
    donneesLignes=file.readlines()
    

In [493]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            donnees.append(ligne)
    return

def donneesNoLine():
    donnees.append("")
    for ligne in donneesLignes:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            donnees[-1]+=ligne+"<br>"
        elif ligne.strip().strip(";")=="":
            donnees.append("")

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
elif not boolEX:
    donneesNoLine()
else:
#    print "sans #EX;"
    donneesNoEX()

In [494]:
if u"Pénalité" in configuration:
    ToolsMoodleXML.penalite=configuration[u"Pénalité"]
if u"NbChoix" in configuration:
    ToolsMoodleXML.maxChoix=configuration["NbChoix"]-1 #maxChoix alternatives à la solution pour avoir NbChoix en tout

In [495]:
categorie=configuration[u"Catégorie"]
titreGeneral=configuration[u"Titre"]
questions=XMLSeries(categorie)
debutConsigne=configuration[u"Consigne"][u"début"]
finConsigne=configuration[u"Consigne"][u"fin"]

In [496]:
def escapeDieseTilde(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)(#|~)",result)
    while m:
        print m.group(0),m.group(1)   
        result=result.replace(m.group(0),ur"\%s"%m.group(1))
        m=re.search(ur"(?<!\\)(#|~)",result)
    return result

def questionParentheses(chaine):
    result=chaine
#    m=re.search(ur"\(([^)]+)\)",result)
    m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    while m:
        result=result.replace(m.group(0),u"{1:SAC:=%s}"%m.group(1))
        m=re.search(ur"(?<!\\)\((.+)(?<!\\)\)",result)
    result=result.replace(ur"\(","(").replace(ur"\)",")")
    return result

def questionCrochets(chaine):
    result=chaine
    m=re.search(ur"(?<!\\)\[(([^\]]|\\\])+)(?<!\\)\]",result)
    while m:
        resultMC="~".join(m.group(1).split(";"))
        print resultMC
        result=result.replace(m.group(0),u"{1:MC:=%s}"%resultMC)
        m=re.search(ur"(?<!\\)\[(([^\]]|\\\])+)(?<!\\)\]",result)
    result=result.replace(ur"\[","[").replace(ur"\]","]")
    return result

def questionParCro(chaine):
    chaineEsc=escapeDieseTilde(chaine)
    chainePar=questionParentheses(chaineEsc)
    chaineCro=questionCrochets(chainePar)
    return chaineCro

In [497]:
print questionParCro(ur"Ceci est [une;un;des] réponse \((courte)\), cela est un [choix multiple;\[schmilblick\];truc~bizarre;bazar;#EX].")

~ ~
# #
une~un~des
choix multiple~\[schmilblick\]~truc\~bizarre~bazar~\#EX
Ceci est {1:MC:=une~un~des} réponse ({1:SAC:=courte}), cela est un {1:MC:=choix multiple~[schmilblick]~truc\~bizarre~bazar~\#EX}.


In [498]:
for numEx,exercices in enumerate(donnees):
    titre="%s%02d"%(titreGeneral, numEx+1)
    corps=debutConsigne+questionParCro(exercices)+finConsigne
    print corps
    questions.addExercice(ClozeExercice(titre,corps))

bibliothèque~parc~bureau de poste~piscine
journaliste~peintre~acteur / actrice~romancier(ère)
Fruits~Leçon~Réveil~Arbre
fleur~voiture~pain~livre~chien
Quoi/que~Nom/prénom~Je/moi~Exemple
Employé(e) d’une compagnie~Metteur en scène~Romancier(ère)~Chanteur(euse) d’opéra
Amie~Qui~Personne~Dialogue
Médecin~Chaise~Bureau/table~Photo
Enchanté(é) de vous rencontrer.~Ah, vraiment ?~Cela fait plaisir de vous revoir.~Bonjour.
대학원생~대학생~사람~선생님
역사~읽기~전공~대화
네~아니요~도~씨
독일~영국~미국~호주
일본어~중국어~일본 사람~가봉 사람
김미진 선생님은 배우예요.~김미진은 선생님이에요.~김미진 씨는 선생님이에요.~김미진 씨는 배우 선생님이에요.
나 : 아니요, 모르간도 프랑스 사람이 아니에요.~나 : 네, 모르간은 프랑스 사람이 아니에요.~나 : 아니요, 모르간은 프랑스 사람이 아니에요.~나 : 아니요, 모르간은 프랑스 사람이에요.
김 선생님은 화가예요 ?~김 선생님, 당신은 화가예요 ?~김 교수님, 당신은 화가예요 ?~김 선생님, 너는 화가예요 ?
괜차나요~괜찬아요~괜찬나요~괜차나요
한구거~한국거~하구거~안구거
꼳~꼿~꽅~꽃
<br/>Choisissez la traduction correcte.<br>
<br>1.	도서관
<br>{1:MC:=bibliothèque~parc~bureau de poste~piscine}<br>
<br>2.	 기자
<br>{1:MC:=journaliste~peintre~acteur / actrice~romancier(ère)}<br>
<br>3.	 과일
<br>{1:MC:=Fruits~Leçon~Révei

In [499]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getSeries())

In [500]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [501]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [502]:
print questionParCro(donnees[0])

bibliothèque~parc~bureau de poste~piscine
journaliste~peintre~acteur / actrice~romancier(ère)
Fruits~Leçon~Réveil~Arbre
fleur~voiture~pain~livre~chien
Quoi/que~Nom/prénom~Je/moi~Exemple
Employé(e) d’une compagnie~Metteur en scène~Romancier(ère)~Chanteur(euse) d’opéra
Amie~Qui~Personne~Dialogue
Médecin~Chaise~Bureau/table~Photo
Enchanté(é) de vous rencontrer.~Ah, vraiment ?~Cela fait plaisir de vous revoir.~Bonjour.
대학원생~대학생~사람~선생님
역사~읽기~전공~대화
네~아니요~도~씨
독일~영국~미국~호주
일본어~중국어~일본 사람~가봉 사람
김미진 선생님은 배우예요.~김미진은 선생님이에요.~김미진 씨는 선생님이에요.~김미진 씨는 배우 선생님이에요.
나 : 아니요, 모르간도 프랑스 사람이 아니에요.~나 : 네, 모르간은 프랑스 사람이 아니에요.~나 : 아니요, 모르간은 프랑스 사람이 아니에요.~나 : 아니요, 모르간은 프랑스 사람이에요.
김 선생님은 화가예요 ?~김 선생님, 당신은 화가예요 ?~김 교수님, 당신은 화가예요 ?~김 선생님, 너는 화가예요 ?
괜차나요~괜찬아요~괜찬나요~괜차나요
한구거~한국거~하구거~안구거
꼳~꼿~꽅~꽃
Choisissez la traduction correcte.<br>
<br>1.	도서관
<br>{1:MC:=bibliothèque~parc~bureau de poste~piscine}<br>
<br>2.	 기자
<br>{1:MC:=journaliste~peintre~acteur / actrice~romancier(ère)}<br>
<br>3.	 과일
<br>{1:MC:=Fruits~Leçon~Réveil~Arb